# 04 A0V calibrators
Let's get a list of the A0V star calibrators for each night of our target observations

In [1]:
#/work/00115/gebhardt/wcs/wcstools-3.9.2/bin/gethead -p /work/03946/hetdex/maverick/LRS2/CALS/*202101*spectra.fits OBJECT | grep 'HR\|HD'

In [2]:
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas
import pandas as pd
import numpy as np

In [3]:
import pyarrow.feather as feather
df = feather.read_feather('../../data/HPF/HAT_P_67_log.feather')

In [4]:
from tqdm import tqdm
from astropy.io import fits

In [5]:
substrings = df['night_of'].value_counts().index.str.replace('-', '')

In [6]:
def get_metadata_from_fits(filenames, dynamic_only=True):
    df_out = pd.DataFrame()
    for file in tqdm(filenames):
        hdu = fits.open(file)
        hdr = hdu[0].header
        
        # Remove duplicate keys like `HISTORY`
        columns = np.array(list(hdr.keys()))
        _, indices, counts = np.unique(columns, return_index=True, return_counts=True)
        columns = columns[np.isin(columns, columns[indices[counts==1]])]
        df = pd.Series({col:hdr[col] for col in columns }).to_frame().T
        df['file']=file
        df_out = df_out.append(df, ignore_index=True)
        hdu.close()
    
    # Only keep columns that change in time
    if dynamic_only:
        dynamic_columns = df_out.apply(lambda x: len(pd.unique(x))) != 1
        df_out = df_out[dynamic_columns[dynamic_columns].index.values]
        
    return df_out

In [7]:
base_string = '/Volumes/pecos/HPF/TACC/CALS/CALS_Goldilocks_only/*{}*spectra.fits'
files_dict = {substr:glob.glob(base_string.format(substr)) for substr in substrings}

In [8]:
{key:len(files_dict[key]) for key in substrings}

{'20200614': 48,
 '20200521': 27,
 '20220428': 55,
 '20200427': 39,
 '20220621': 48,
 '20200426': 39,
 '20220429': 51,
 '20220430': 44,
 '20220501': 54,
 '20200617': 27,
 '20200615': 55,
 '20200613': 6,
 '20200523': 27,
 '20200522': 27,
 '20200520': 42,
 '20200519': 27,
 '20200428': 27,
 '20220427': 55,
 '20200612': 28,
 '20220622': 55,
 '20220619': 129,
 '20220712': 68,
 '20210330': 32,
 '20210225': 33,
 '20210131': 41,
 '20210303': 37,
 '20210130': 35,
 '20200731': 37,
 '20200721': 27,
 '20220728': 56,
 '20220719': 60,
 '20220715': 54,
 '20220714': 57,
 '20210223': 35,
 '20220710': 55,
 '20220709': 46,
 '20220630': 53,
 '20220625': 30,
 '20220729': 51}

In [141]:
substrings

39

In [139]:
tell_matches = {'20220430':'HR4943_tell_std', 
                '20220621':'HR7632_tell_std', 
                '20220712':'HR5023_tell_std',
                '20220501':'HR7803_tell_std',
                '20200615':['HR8487','HR6977'],
                '20210330':'HR4137_tell_std',
                '20220622':'HR4828_tell_std',
                '20210303':'HR2893_tell_std',
                '20210130':'HR2893',
                '20200731':'HR7634_tell_std',
                '20220728':'HR5532_tell_std',
                '20220719':'HR7628_tell_std',
                '20220715':'HR8186_tell_std',
               '20220714':'HR5373_tell_std',
               '20210223':'HR4829_tell_std',
               '20220710':'HR5373_tell_std',
               '20220709':'HR189_tell_std',
               '20220625':'HR189_tell_std',
               '20220729':'HR6195_tell_std'}

In [140]:
len(tell_matches)

19

In [135]:
df[df.night_of=='2022-06-22']

,night_number,night_of,local_hour,ORIGIN,DATE,OBJECT,OBSERVER,OBSTYPE,DATE-OBS,CAL-OBJ,...,JD,BTJD,phase,normalized_phase,in_transit_flag,campaign,stddev_order19,RV_planet,track,in_transit_text
135,1666,2022-06-22,3.46,astropy.fits,2022-06-23T08:27:12,HAT-P-67b_EPOCH3,CFB,Sci,2022-06-23T08:27:34.605231,dark,...,2.459754e+06,2753.852484,2.107044,0.438046,False,P4,0.070898,55.835816,west,
136,1666,2022-06-22,3.56,astropy.fits,2022-06-23T08:32:58,HAT-P-67b_EPOCH3,CFB,Sci,2022-06-23T08:33:26.392145,dark,...,2.459754e+06,2753.856555,2.111115,0.438892,False,P4,0.076238,55.111086,west,
137,1666,2022-06-22,3.65,astropy.fits,2022-06-23T08:38:45,HAT-P-67b_EPOCH3,CFB,Sci,2022-06-23T08:39:07.531528,dark,...,2.459754e+06,2753.860504,2.115064,0.439713,False,P4,0.070603,54.406802,west,


In [136]:
df_raw[df_raw.OBJECT=='HR4828_tell_std']

,DATE,OBJECT,OBSTYPE,DATE-OBS,SCI-OBJ,CAL-OBJ,SKY-OBJ,QTIME,QRA,QDEC,...,AVRNV,ADSUBV,AVRF3P0V,NREADS,ITIME,NONDR,EXPLNDR,BRVCORR,LRVCORR,file
0,2022-06-23T03:39:33,HR4828_tell_std,Sci,2022-06-23T03:39:55.876815,Sci,dark,Sky,1655955565.259712393,12:41:53.00,+10:14:08.00,...,0.120456,0.531867,3.028669,11,117.15,11,117.15,-28608.425762,-29.269848,/Volumes/pecos/HPF/TACC/CALS/CALS_Goldilocks_o...


In [138]:
files = glob.glob(base_string.format('20220713'))
#files = files_dict['20200613']
df_raw = get_metadata_from_fits(files)
df_raw['OBJECT'].value_counts()

100%|███████████████████████████████████████████| 54/54 [00:00<00:00, 84.00it/s]


Quartz FCU          14
LFC Cal             10
Alpha Bright Cal     8
LFC FCU              8
UNe Slave Cal        6
Etalon Cal           5
Dark                 2
HR5023_tell_std      1
Name: OBJECT, dtype: int64